In [7]:
import itertools
import functools
import collections
import pathlib
from pprint import pprint

from structure import Structure

# MUST have top level in structure
TEST_STRUCTURE = {
    "separator": "/",
    "structure": {
        "project": ["euphoria"],
        "sections": {
            # "views": ["frontend", "backend", "model", "view"],
            # "routes": ["baseroutes", "v2routes"],
            # "project": ["euphoria"],
            "program": ["supervisor", "nginx"],
            "resource": ["app", "api"],
            "env": ["dev", "test", "prod"],
            "sections": {"suffix": ["run.conf", "startup.conf", "destroy.conf"]},
        },
        },
    # },
}


s = Structure(**TEST_STRUCTURE)
# s.display()




In [9]:
s.display_tree_groups('project/env/resource/program/suffix')

'project/env/resource/program/suffix'

└── euphoria
    ├── test
    │   ├── app
    │   │   ├── nginx
    │   │   │   ├── startup.conf
    │   │   │   ├── destroy.conf
    │   │   │   └── run.conf
    │   │   └── supervisor
    │   │       ├── startup.conf
    │   │       ├── destroy.conf
    │   │       └── run.conf
    │   └── api
    │       ├── nginx
    │       │   ├── startup.conf
    │       │   ├── destroy.conf
    │       │   └── run.conf
    │       └── supervisor
    │           ├── startup.conf
    │           ├── destroy.conf
    │           └── run.conf
    ├── prod
    │   ├── app
    │   │   ├── nginx
    │   │   │   ├── startup.conf
    │   │   │   ├── destroy.conf
    │   │   │   └── run.conf
    │   │   └── supervisor
    │   │       ├── startup.conf
    │   │       ├── destroy.conf
    │   │       └── run.conf
    │   └── api
    │       ├── nginx
    │       │   ├── startup.conf
    │       │   ├── destroy.conf
    │       │   └── run.conf
    │       └── supervis

In [10]:
from pathlib import Path

# prefix components:
SPACE =  '    '
BRANCH = '│   '
# pointers:
TEE =    '├── '
LAST =   '└── '


def tree(dir_path: Path, prefix: str=''):
    contents = list(dir_path.iterdir())
    # contents each get pointers that are ├── with a final └── :
    pointers = [TEE] * (len(contents) - 1) + [LAST]
    for pointer, path in zip(pointers, contents):
        yield prefix + pointer + path.name
        if path.is_dir(): # extend the prefix and recurse:
            extension = BRANCH if pointer == TEE else SPACE 
            # i.e. SPACE because last, └── , above so no more |
            yield from tree(path, prefix=prefix+extension)


for line in tree(Path('.')):
    print(line)

├── test_structure.py
├── pyproject.toml
├── __pycache__
│   └── structure.cpython-310.pyc
├── euphoria
│   ├── euphoria.app.nginx.dev.conf
│   ├── app
│   │   ├── nginx
│   │   │   ├── prod
│   │   │   │   └── conf
│   │   │   ├── test
│   │   │   │   └── conf
│   │   │   └── dev
│   │   │       └── conf
│   │   └── supervisor
│   │       ├── prod
│   │       │   └── conf
│   │       ├── test
│   │       │   └── conf
│   │       └── dev
│   │           └── conf
│   ├── euphoria.api.supervisor.dev.conf
│   ├── euphoria.app.supervisor.prod.conf
│   ├── euphoria.api.nginx.prod.conf
│   ├── euphoria.app.nginx.test.conf
│   ├── euphoria.app.supervisor.dev.conf
│   ├── euphoria.api.supervisor.test.conf
│   ├── api
│   │   ├── nginx
│   │   │   ├── prod
│   │   │   │   └── conf
│   │   │   ├── test
│   │   │   │   └── conf
│   │   │   └── dev
│   │   │       └── conf
│   │   └── supervisor
│   │       ├── prod
│   │       │   └── conf
│   │       ├── test
│   │       │   └── conf
│   │      

In [49]:
TREE_GROUP

{'euphoria': {'supervisor': {'app': {'prod': {'conf': {}},
    'dev': {'conf': {}},
    'test': {'conf': {}}},
   'api': {'prod': {'conf': {}}, 'dev': {'conf': {}}, 'test': {'conf': {}}}},
  'nginx': {'app': {'prod': {'conf': {}},
    'dev': {'conf': {}},
    'test': {'conf': {}}},
   'api': {'prod': {'conf': {}}, 'dev': {'conf': {}}, 'test': {'conf': {}}}}}}

In [48]:
from pathlib import Path

TREE_GROUP = s.tree_groups['project.program.resource.env.suffix']
# print(TREE_GROUP)


def _create_tree_structure(tree_group: dict, prefix: str=''):
    # prefix components:
    SPACE =  '    '
    BRANCH = '│   '
    # pointers:
    TEE =    '├── '
    LAST =   '└── '
    # contents each get pointers that are ├── with a final └── :
    pointers = [TEE] * (len(tree_group) - 1) + [LAST]
    # print(f'{pointers=}')
    for pointer, section in zip(pointers, tree_group):
        yield prefix + pointer + section
        # print(f'{tree_group=}')
        if isinstance(tree_group.get(section), dict): # extend the prefix and recurse:
            extension = BRANCH if pointer == TEE else SPACE 
            # print(f'{extension=}')
            # i.e. SPACE because last, └── , above so no more |
            yield from _create_tree_structure(tree_group[section], prefix=prefix+extension)


for line in _create_tree_structure(TREE_GROUP):
    print(line)

└── euphoria
    ├── supervisor
    │   ├── app
    │   │   ├── prod
    │   │   │   └── conf
    │   │   ├── dev
    │   │   │   └── conf
    │   │   └── test
    │   │       └── conf
    │   └── api
    │       ├── prod
    │       │   └── conf
    │       ├── dev
    │       │   └── conf
    │       └── test
    │           └── conf
    └── nginx
        ├── app
        │   ├── prod
        │   │   └── conf
        │   ├── dev
        │   │   └── conf
        │   └── test
        │       └── conf
        └── api
            ├── prod
            │   └── conf
            ├── dev
            │   └── conf
            └── test
                └── conf


In [43]:
len(TREE_GROUP)

1

In [44]:
TREE_GROUP.get('euphoria')

{'supervisor': {'app': {'prod': {'conf': {}},
   'dev': {'conf': {}},
   'test': {'conf': {}}},
  'api': {'prod': {'conf': {}}, 'dev': {'conf': {}}, 'test': {'conf': {}}}},
 'nginx': {'app': {'prod': {'conf': {}},
   'dev': {'conf': {}},
   'test': {'conf': {}}},
  'api': {'prod': {'conf': {}}, 'dev': {'conf': {}}, 'test': {'conf': {}}}}}